<a href="https://colab.research.google.com/github/felipeserna/holbertonschool-machine_learning/blob/master/supervised_learning/0x09-transfer_learning/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python3
"""
Script that trains a convolutional neural network
to classify the CIFAR 10 dataset.
In the same file, write a function 'def preprocess_data(X, Y):'
that pre-processes the data for your model
"""


import tensorflow.keras as K


def preprocess_data(X, Y):
    """
    Returns: X_p, Y_p
    """
    # Preprocessing needed in each Keras Application
    X_p = K.applications.densenet.preprocess_input(X)
    # Converts a label vector into a one-hot matrix
    Y_p = K.utils.to_categorical(Y, num_classes=10)

    return X_p, Y_p


if __name__ == "__main__":
    # Dataset of 50,000 32x32 color training images and 10,000 test images,
    # labeled over 10 categories
    (x_train, y_train), (X, Y) = K.datasets.cifar10.load_data()

    # preprocessing
    x_train_p, y_train_p = preprocess_data(x_train, y_train)
    x_test_p, y_test_p = preprocess_data(X, Y)

    # DenseNet121
    #  loads weights pre-trained on ImageNet
    dense_121 = K.applications.DenseNet121(weights='imagenet',
                                           include_top=False,
                                           input_shape=(224, 224, 3))

    dense_121.trainable = False

    input = K.Input(shape=(32, 32, 3))
    # lambtha layer scales up the data to the correct size
    lambtha = K.layers.Lambda(lambda X: K.backend.resize_images(X, 7, 7,
                              data_format="channels_last",
                              interpolation='bilinear'))(input)
    output = dense_121(lambtha, training=False)
    output = K.layers.Flatten()(output)
    output = K.layers.Dense(512, activation='relu')(output)
    output = K.layers.Dropout(0.2)(output)
    output = K.layers.Dense(256, activation='relu')(output)
    output = K.layers.Dropout(0.2)(output)
    output = K.layers.Dense(128, activation='relu')(output)
    output = K.layers.Dropout(0.2)(output)
    output = K.layers.Dense(10, activation='softmax')(output)

    model = K.Model(input, output)

    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['acc'])

    # training
    history = model.fit(x=x_train_p, y=y_train_p,
                        batch_size=128, epochs=5,
                        validation_data=(x_test_p, y_test_p),
                        verbose=1)

    model.save('cifar10.h5')

29089792/29084464 [==============================] - 0s 0us/step
Epoch 1/5
391/391 [==============================] - 203s 422ms/step - loss: 0.7992 - acc: 0.7879 - val_loss: 0.3495 - val_acc: 0.8857
Epoch 2/5
391/391 [==============================] - 164s 420ms/step - loss: 0.3668 - acc: 0.8832 - val_loss: 0.3228 - val_acc: 0.8961
Epoch 3/5
391/391 [==============================] - 164s 419ms/step - loss: 0.2815 - acc: 0.9102 - val_loss: 0.3125 - val_acc: 0.9053
Epoch 4/5
391/391 [==============================] - 163s 418ms/step - loss: 0.2331 - acc: 0.9245 - val_loss: 0.3053 - val_acc: 0.9054
Epoch 5/5
391/391 [==============================] - 164s 419ms/step - loss: 0.1972 - acc: 0.9348 - val_loss: 0.3181 - val_acc: 0.9054


In [2]:
# to fix issue with saving keras applications
K.learning_phase = K.backend.learning_phase 

_, (X, Y) = K.datasets.cifar10.load_data()
X_p, Y_p = preprocess_data(X, Y)
model = K.models.load_model('cifar10.h5')
model.evaluate(X_p, Y_p, batch_size=128, verbose=1)

79/79 [==============================] - 30s 356ms/step - loss: 0.3181 - acc: 0.9054


[0.31812506914138794, 0.9053999781608582]